In [184]:
# Data Science
import re
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from pandasgui import show
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

In [7]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [18]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\MASTER_7_14.csv")

In [19]:
matches

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0,NaN,1521494.0,0,NaN,['cerebral']
1,1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0,NaN,1012865.0,0,NaN,['memory']
2,2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,...,[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416.0,0,NaN,['dementia']
3,3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0,1002888.0,1746848.0,0,NaN,"['cognition', 'memory', 'moca']"
4,4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0,1002888.0,1746848.0,0,NaN,"['cognition', 'memory']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279219,279219,60817,60817,277874,NaN,113705759,10127549029,Z16320629,3.285425e+09,2020-04-28 00:00:00.0000000,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(538, 547), (1626, 1635), (1844, 1853), (108,...","------- se ""she took a break"" and wants to dis...",1949,11314.0,1007065.0,1558180.0,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279220,279220,60818,60818,277883,NaN,113705759,10127549029,Z16320629,3.305416e+09,2020-09-22 00:00:00.0000000,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(323, 332), (1411, 1420), (1629, 1638), (108,...","------- all of her medications, including asp...",1734,10113.0,1007065.0,1558180.0,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279221,279221,60819,60819,277896,NaN,113705759,10127549029,Z16320629,3.327430e+09,2021-01-25 00:00:00.0000000,...,"[' cognition', ' cerebral', ' memory', ' cereb...","[(2816, 2824), (108, 117), (326, 335), (974, 9...","------- ? Patient, Chart, daughter Olga Mode o...",3009,12320.0,1007065.0,1558180.0,0,NaN,"['dementia', 'cognition', 'memory', 'mci', 'ce..."
279222,279222,60820,60820,277912,NaN,113705759,10127549029,Z16320629,3.349755e+09,2021-04-20 00:00:00.0000000,...,"[' cognition', ' cerebral', ' cognitive impair...","[(2103, 2111), (108, 117), (326, 335), (544, 5...","------- ? Patient, Chart, daughter Olga Mode o...",2835,11250.0,1007065.0,1558180.0,0,NaN,"['dementia', 'cognition', 'moca', 'cognitive i..."


In [20]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [29]:
keywords = ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

In [86]:
subset.append(pd.DataFrame(matches.iloc[64072]).T)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
64072,64072,13505,16807,16807,71543.0,100551185,10069628096,Z10137763,3210780793.0,2018-08-08 00:00:00.0000000,...,[' memory'],"[(3170, 3563), (5054, 5447)]",------- ts healing well from radiation. Denie...,804,10754.0,1005782.0,1009871.0,0,NaN,['memory']


In [21]:
TOTAL_KEYWORD_MATCHES = list(total["Total_Count"].unique())[1]

In [174]:
diverse_df = pd.DataFrame(columns = list(matches.columns))

In [175]:
percent = 5000 / TOTAL_KEYWORD_MATCHES

int(math.ceil(24 * percent))

1

In [191]:
def diversity_sampling(SEQ_COUNT):
    global diverse_df
    # calculating percent that will be taken from each set of keyword matches
    percent = SEQ_COUNT / TOTAL_KEYWORD_MATCHES
    
    for word in tqdm(keywords):
        # getting subset of dataframe that contains sequence with keyword match
        contains = matches['padded_regex_match'].str.contains(str(word))
        filtered = matches[contains]
        
        # getting number of matches to take from subset of keyword matches
        take = int(math.ceil(len(filtered) * percent))
        if (take == 1):
            take = 2
        subset = pd.DataFrame(columns = list(matches.columns))  
        
        # generating random index numbers for subset
        for i in range(take):
            idx = random.randrange(0, len(filtered))
            subset = subset.append(pd.DataFrame(matches.iloc[idx]).T)
            #print(len(subset))  
            
        diverse_df = diverse_df.append(subset)

In [192]:
diversity_sampling(5000)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:15<00:00,  1.15it/s]


In [193]:
len(diverse_df["EMPI"].unique())

3028

In [194]:
diverse_df = diverse_df.reset_index()
diverse_df

,level_0,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,0,24365.0,24365,24365,30123,30123,128460.0,100129465,5114654,Z6923710,...,[' memory'],"(3296, 4092)","neuralgia, didn't tolerate oxcarbazepin, bacl...",796,4295.0,1009470.0,1007107.0,0,NaN,['memory']
1,1,47205.0,47205,47210,58614,58614,255087.0,100335779,10020477146,Z8490045,...,[' cognition'],"(3172, 3971)",nicteric. [Respiratory]: (+) Slight crackles a...,799,4597.0,1000899.0,1000899.0,1,NaN,['cognition']
2,2,9334.0,9334,9334,11514,11514,48761.0,100053905,10041789552,Z6458881,...,[' memory'],"(304, 1104)",e. Patient requires physical assistance for pu...,800,1105.0,1000364.0,NaN,0,NaN,['memory']
3,3,12033.0,12033,12033,14809,14809,62634.0,100067895,62090,Z6522871,...,[' dementia'],"(6782, 7580)",Emotionally abused: Not on file Physically ab...,798,10968.0,1009687.0,1672366.0,0,NaN,['dementia']
4,4,40298.0,40298,40300,49933,49933,215342.0,100249188,10052853628,Z7830185,...,[' memory'],"[(3762, 4152), (5217, 5610)]",------- mouth nightly. gabapentin (NEURONTIN...,801,12097.0,1007039.0,1007039.0,1,NaN,['memory']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,1743,NaN,1743,1743,2144,2144,8293.0,100013241,10040207135,Z6363555,...,[' cognition'],"[(6273, 6663), (20556, 20952)]","------- Sig: Reusable underpads, large. Heigh...",804,21432.0,1013005.0,NaN,0,NaN,['cognition']
10014,1098,NaN,1098,1098,1340,1340,5380.0,100009864,10040121179,Z6360622,...,[' memory'],"(181, 975)",sgeneral.org/gastroenterology Physician: Dr. M...,794,4197.0,1001934.0,NaN,0,NaN,['memory']
10015,142,NaN,142,142,169,169,546.0,100001873,10040014499,Z6353764,...,[' moca'],"(0, 256)",----------------------------------------------...,256,257.0,1004526.0,1117454.0,0,NaN,['moca']
10016,203,NaN,203,203,239,239,813.0,100002621,10040020207,Z6354398,...,[' cognition'],"(15505, 16300)",edicine Very bad pain not relieved by your pai...,795,16503.0,1002622.0,NaN,0,NaN,['cognition']


In [187]:
diverse_df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_sample_7_19.csv")

In [179]:
# defining summary stats df
cols =  ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

summary_stats_seq  = pd.DataFrame(pd.np.empty((len(diverse_df), len(cols) + 3)) * pd.np.nan) 
summary_stats_seq.columns = ['EMPI', 'EncounterID', 'NoteID'] + list(cols)
summary_stats_seq["EMPI"] = diverse_df["EMPI"].to_list()
summary_stats_seq["EncounterID"] = diverse_df["PatientEncounterID"].to_list()
summary_stats_seq["NoteID"]  = diverse_df["NoteID"].to_list()

In [180]:
for i in tqdm(range(len(diverse_df))):
    match_tuples = []
    keyword_list = []
    
    keyword_list = list(eval(diverse_df['padded_regex_match'][i]))
    for z in range(len(keyword_list)):
        match_tuples.append((keyword_list[z], 0))

    freq = {"dementia":0
        ,"cognition":0
        ,"cognition":0
        ,"memory":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
        ,"ad": 0
        ,"lbd": 0
    }

    for k, v in match_tuples:   
        if (k != ''):
            if (k == 'cognitive impairmentcognitiveimpairment' or k == 'cognitiveimpairmentcognitive impairment'):
                k = 'cognitive impairment'
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            else:
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            
    #print(freq.items())
    #print(summary_stats["dementia"])
    for k, v in freq.items():
        #print(str(k.lower()))
        summary_stats_seq[str(k.lower())][i] = int(v)

100%|█████████████████████████████████████████████████████████████████████████████| 5009/5009 [00:39<00:00, 127.16it/s]


In [181]:
summary_stats_seq

,EMPI,EncounterID,NoteID,dementia,cognition,memory,mmse,moca,alzheimer,cognitive impairment,...,cerebellar,neurocognitive,lewy,pick's,corticobasal,cerebral,cerebrovascular,amnesia,ad,lbd
0,100129465,3.121598e+09,1096403210,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100335779,3.224430e+09,2294910337,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100053905,3.176475e+09,1736629435,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100067895,3.243889e+09,2415533763,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100249188,3.143184e+09,1489471706,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,100014582,3.202998e+09,2001546644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5005,100011780,3.259835e+09,2548501334,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5006,100000920,3.153583e+09,1484554679,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5007,100001150,3.121057e+09,1068603135,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
show(summary_stats_seq)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI...
PandasGUI INFO — pandasgui.gui — Opening PandasGUI...
PandasGUI INFO — pandasgui.gui — Opening PandasGUI...


In [186]:
summary_stats_seq.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_seqs_7_19.csv")

In [195]:
slat_excel = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_19.xlsx")

In [196]:
slat_excel

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6923710,5114654,100129465,3121597648,Progress Notes,1096403210,1045849581,2016-06-20,----------------------------------------------...,""" neuralgia, didn't tolerate oxcarbazepin, bac...","""['memory']"""
1,Z8490045,10020477146,100335779,3224429970,Progress Notes,2294910337,2263432565,2019-01-16,----------------------------------------------...,"""nicteric. [Respiratory]: (+) Slight crackles ...","""['cognition']"""
2,Z6458881,10041789552,100053905,3176475339,Administrative,1736629435,1687145674,2017-10-26,----------------------------------------------...,"""e. Patient requires physical assistance for p...","""['memory']"""
3,Z6522871,62090,100067895,3243889319,Progress Notes,2415533763,2385243280,2019-04-15,----------------------------------------------...,""" Emotionally abused: Not on file Physically a...","""['dementia']"""
4,Z7830185,10052853628,100249188,3143184158,MR AVS Snapshot,1489471706,1433162574,2017-04-24,----------------------------------------------...,""" ------- mouth nightly. gabapentin (NEURONTI...","""['memory']"""
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6364707,10040240250,100014582,3202997929,Consults,2001546644,1959756720,2018-05-29,----------------------------------------------...,"""cemaker 08/23/2016 \u2022 Presence of prosthe...","""['cerebrovascular']"""
5005,Z6362262,10040169418,100011780,3259835449,Consults,2548501334,2521082423,2019-07-11,----------------------------------------------...,""" ------- , Needs assistance with Mobility Act...","""['cognition', 'memory']"""
5006,Z6352946,204859,100000920,3153583030,Progress Notes,1484554679,1428096733,2017-04-19,----------------------------------------------...,"""7 for a 30 day supply. Patient's granddaughte...","""['memory']"""
5007,Z6353136,10027033512,100001150,3121057435,ED Provider Notes,1068603135,1019356748,2016-06-07,----------------------------------------------...,""" ST Elevations: No Other Finding(s): Non-spec...","""['dementia']"""


In [197]:
slat_excel.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_20.csv", index = False)

In [199]:
## JSON format is incorrect, so rerunning regex matches
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [215]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [213]:
string = (slat_excel["regex_sent"][5]) + " mCi."

print(string)

for p in p_list:
    for match in re.finditer(p, string):
        s = match.start()
        e = match.end()
        print ('String match "%s" at %d:%d' % (string[s:e], s, e))

" ------- nimal, peak 17 mmHg, mean 9 \u2022 Arteriosclerotic heart disease 3/5/2003 Coronary artery disease; imaging neg. 8/02 \u2022 Benign prostatic hyperplasia 3/5/2003 Benign prostatic hypertrophy; Bx (-) BMC 4/06; in trial of finasteride vs. placebo \u2022 Cerebrovascular accident (CVA) \u2022 Cerebrovascular disease 4/18/2007 Cerebrovascular disease \u2022 Coronary artery disease \u2022 History of atrial flutter 10/2012 s/p catheter ablation 2/3/14  -------  Arteriosclerotic heart disease \u2022 Uncoded H/O depression \u2022 Herpes zoster \u2022 Postherpetic neuralgia \u2022 Amnesia \u2022 Cerebrovascular disease \u2022 Uncoded ? Transient ischemic attack \u2022 Nonrheumatic aortic valve stenosis \u2022 Uncoded Falls \u2022 Malaise and fatigue \u2022 Hypercalcemia \u2022 Chronic maxillary sinusitis \u2022 Prostate cancer \u2022 Frequent Medical Issues \u2022 Sleep apnea \u2022 Erectile d" mCi.
re.compile('\\bmemory\\b', re.IGNORECASE)
String match "Cerebrovascular" at 263:278
St

In [223]:
locations = []

for i in tqdm(range(len(slat_excel['regex_sent']))):
    r = str(slat_excel['regex_sent'][i])
    l = []
    for p in p_list:
        for match in re.finditer(p, r):
            l.append(match.group().lower())
    #print(l)
    locations.append(l)
        
slat_excel["regex_match"] = locations

100%|████████████████████████████████████████████████████████████████████████████| 5009/5009 [00:02<00:00, 2235.60it/s]


In [225]:
slat_excel['regex_match'] = slat_excel['regex_match'].apply(json.dumps)

In [226]:
slat_excel

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6923710,5114654,100129465,3121597648,Progress Notes,1096403210,1045849581,2016-06-20,----------------------------------------------...,""" neuralgia, didn't tolerate oxcarbazepin, bac...","[""memory""]"
1,Z8490045,10020477146,100335779,3224429970,Progress Notes,2294910337,2263432565,2019-01-16,----------------------------------------------...,"""nicteric. [Respiratory]: (+) Slight crackles ...","[""cognition""]"
2,Z6458881,10041789552,100053905,3176475339,Administrative,1736629435,1687145674,2017-10-26,----------------------------------------------...,"""e. Patient requires physical assistance for p...","[""memory""]"
3,Z6522871,62090,100067895,3243889319,Progress Notes,2415533763,2385243280,2019-04-15,----------------------------------------------...,""" Emotionally abused: Not on file Physically a...","[""dementia""]"
4,Z7830185,10052853628,100249188,3143184158,MR AVS Snapshot,1489471706,1433162574,2017-04-24,----------------------------------------------...,""" ------- mouth nightly. gabapentin (NEURONTI...","[""memory""]"
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6364707,10040240250,100014582,3202997929,Consults,2001546644,1959756720,2018-05-29,----------------------------------------------...,"""cemaker 08/23/2016 \u2022 Presence of prosthe...","[""cerebrovascular"", ""cerebrovascular""]"
5005,Z6362262,10040169418,100011780,3259835449,Consults,2548501334,2521082423,2019-07-11,----------------------------------------------...,""" ------- , Needs assistance with Mobility Act...","[""cognition"", ""cognition"", ""memory""]"
5006,Z6352946,204859,100000920,3153583030,Progress Notes,1484554679,1428096733,2017-04-19,----------------------------------------------...,"""7 for a 30 day supply. Patient's granddaughte...","[""memory""]"
5007,Z6353136,10027033512,100001150,3121057435,ED Provider Notes,1068603135,1019356748,2016-06-07,----------------------------------------------...,""" ST Elevations: No Other Finding(s): Non-spec...","[""dementia""]"


In [235]:
slat_excel.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\5000_SLAT_import_7_20_v4.csv", index = False)

In [228]:
slat_excel["ContactDTS"] = pd.to_datetime(slat_excel["ContactDTS"], format='%Y-%m-%d')

In [230]:
example = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\regex_dataset.csv")

In [232]:
example.isna().sum()

PatientID                 0
MRN                     275
EMPI                    138
PatientEncounterID      670
InpatientNoteTypeDSC    931
NoteID                    0
NoteCSNID                 0
ContactDTS                0
NoteTXT                   0
regex_sent                0
regex_match               0
dtype: int64

In [233]:
slat_excel.isna().sum()

PatientID                0
MRN                      0
EMPI                     0
PatientEncounterID       0
InpatientNoteTypeDSC    41
NoteID                   0
NoteCSNID                0
ContactDTS               0
NoteTXT                  0
regex_sent               0
regex_match              0
dtype: int64